In [1]:
import cv2
import os

# Path of the video file
video_path = "/content/cctv_video.mp4"

# Create a directory to save the key frames
if not os.path.exists("key_frames"):
  os.makedirs("key_frames")

# Set the threshold for SAD score
threshold = 100000

# Open the video file
vidcap = cv2.VideoCapture(video_path)

# Get the frame rate of the video
fps = vidcap.get(cv2.CAP_PROP_FPS)

# Read the first frame
success, prev_frame = vidcap.read()
prev_frame = cv2.resize(prev_frame, (0,0), fx=360/prev_frame.shape[0], fy=360/prev_frame.shape[0])

# Set the initial frame number to zero
frame_num = 0

# Initialize variables to keep track of the frame with the least motion blur
min_variance = float('inf')
min_variance_frame = None

# Loop through the frames
while success:

  # Read the next frame
  success, curr_frame = vidcap.read()

  if success:
    # Resize the current frame to a height of 360 pixels
    curr_frame = cv2.resize(curr_frame, (0,0), fx=360/curr_frame.shape[0], fy=360/curr_frame.shape[0])

    # Calculate the SAD score between the current and previous frames
    sad_score = cv2.absdiff(curr_frame, prev_frame).sum()

    # Check if the SAD score exceeds the threshold
    if sad_score > threshold:

      # Calculate the Laplacian variance of the current frame
      gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
      lap_var = cv2.Laplacian(gray, cv2.CV_64F).var()

      # Update the minimum variance and the corresponding frame if necessary
      if lap_var < min_variance:
        min_variance = lap_var
        min_variance_frame = curr_frame

      # Save the key frame
      cv2.imwrite("key_frames/key_frame_%d.jpg" % frame_num, curr_frame)

    # Update the previous frame
    prev_frame = curr_frame.copy()

    # Increment the frame number
    frame_num += 1

# Save the frame with the least motion blur as the final key frame
cv2.imwrite("best_key_frame.jpg", min_variance_frame)


True